In [31]:
# ============================================
# 3️ FEATURE ENGINEERING Y SCORING
# ============================================
import pandas as pd
import numpy as np

# Cargar datos preprocesados
df = pd.read_csv("data_clean.csv")
print(f"Datos cargados: {len(df)} registros")
display(df.head())

Datos cargados: 292 registros


,id_paciente,nombre,rut,Edad,fecha_registro,id_medico,FamiliarPrimerGradoCC,FamiliarSegundoGradoCC,DiagnosticoPrevioCancer,Ejercicio,Alcohol,Mamografia,Menstruacion,PrimerHijo,riesgo,Edad_Original
0,926,Bridie,53581753,3,09/10/2025,1,0.0,1,1,1,1.0,0,NaN,1.0,Alto,79
1,109,Bellina,88983746,0,09/10/2025,1,0.0,0,0,2,2.0,0,NaN,1.0,Alto,29
2,295,Starla,2366096,2,09/10/2025,1,1.0,0,0,2,0.0,0,NaN,NaN,Alto,54
3,976,Celine,90902307,3,09/10/2025,1,1.0,1,1,2,2.0,0,NaN,0.0,Moderado,79
4,735,Pietra,66433625,3,09/10/2025,1,1.0,1,1,1,2.0,1,NaN,1.0,Bajo,61


In [27]:
def calcular_puntaje(fila):
    puntaje = 0

    # Asegurarse de que NaN se traten como 0 (valor neutro)
    def safe_int(value):
        try:
            return int(value)
        except:
            return 0

    # Edad
    edad_idx = safe_int(fila.get('Edad', 0))
    puntaje += [0, 5, 12, 20][min(edad_idx, 3)]

    # Familiares
    puntaje += 25 if fila.get('FamiliarPrimerGradoCC', 0) == 1 else 0
    puntaje += 4 if fila.get('FamiliarSegundoGradoCC', 0) == 1 else 0

    # Diagnóstico previo
    puntaje += 30 if fila.get('DiagnosticoPrevioCancer', 0) == 1 else 0

    # Menstruación
    men_idx = safe_int(fila.get('Menstruacion', 0))
    puntaje += [6, 3, 0][min(men_idx, 2)]

    # Primer hijo
    hijo_idx = safe_int(fila.get('PrimerHijo', 0))
    puntaje += [0, 5, 8][min(hijo_idx, 2)]

    # Ejercicio
    ej_idx = safe_int(fila.get('Ejercicio', 0))
    puntaje += [5, 3, -2, -5][min(ej_idx, 3)]

    # Alcohol
    alc_idx = safe_int(fila.get('Alcohol', 0))
    puntaje += [0, 2, 5, 8][min(alc_idx, 3)]

    # Mamografía
    puntaje += -3 if fila.get('Mamografia', 0) == 1 else 0

    return puntaje

# Calcular puntaje sin fallar por NaN
df['Puntaje'] = df.apply(calcular_puntaje, axis=1)
print("Puntajes calculados correctamente.")

Puntajes calculados correctamente.


In [28]:
def clasificar_riesgo(puntaje):
    if puntaje < 20:
        return 0  # Bajo
    elif puntaje < 50:
        return 1  # Moderado
    else:
        return 2  # Alto

df['Riesgo'] = df['Puntaje'].apply(clasificar_riesgo)

In [29]:
print("\nEjemplo de puntajes y riesgo asignado:")
display(df[['Edad', 'Puntaje', 'Riesgo']].head())

print("\nDistribución de categorías de riesgo:")
print(df['Riesgo'].value_counts().sort_index())


Ejemplo de puntajes y riesgo asignado:


,Edad,Puntaje,Riesgo
0,3,70,2
1,0,14,0
2,2,41,1
3,3,88,2
4,3,95,2



Distribución de categorías de riesgo:
Riesgo
0     18
1    100
2    174
Name: count, dtype: int64


In [30]:
df.to_csv("data_scored.csv", index=False)
print("Archivo enriquecido guardado como 'data_scored.csv'")

Archivo enriquecido guardado como 'data_scored.csv'
